###  Getting Started with Pyspark

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import HiveContext
from pyspark.sql import functions as F
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

import numpy
import pandas as pd
from pandas import DataFrame as df
import difflib
import datetime

#sc is the sparkContext that gets created whe pyspark is started
hc = HiveContext(sc)

#### If SparkContext doesnot get Intialized by itself, following set of commands may be used:

import findspark <br>
findspark.init() <br>
import pyspark # only run after findspark.init() <br>
from pyspark.sql import SparkSession <br>
spark = SparkSession.builder.getOrCreate() <br>
import pandas as pd <br>
sc = spark.sparkContext <br>

###  Make a sample dataframe

#### 1. Using pandas Dataframe:

Read Data into Pandas from csv, txt, excel files, etc. <br> 
Define a Schema: columns and their data types <br>
Use CreateDataFrame() to convert to spark dataframe

In [12]:
data1 = {'PassengerId': {0: 1, 1: 2, 2: 3, 3: 4, 4: 5},
         'Name': {0: 'Owen', 1: 'Florence', 2: 'Laina', 3: 'Lily', 4: 'William'},
         'Gender': {0:'M', 1: 'F', 2: 'F', 3: 'F', 4: 'M'},
         'Survived': {0: 0, 1: 1, 2: 1, 3: 1, 4: 0}}

data2 = {'PassengerId': {0: 1, 1: 2, 2: 3, 3: 4, 4: 5},
         'Age': {0: 22, 1: 38, 2: 26, 3: 35, 4: 35},
         'Fare': {0: 7.3, 1: 71.3, 2: 7.9, 3: 53.1, 4: 8.0},
         'Pclass': {0: 3, 1: 1, 2: 3, 3: 1, 4: 3}}

df1_pd = pd.DataFrame(data1, columns=data1.keys())
df2_pd = pd.DataFrame(data2, columns=data2.keys())

#### Without defining Data Schema

In [13]:
df1 = hc.createDataFrame(df1_pd)
df2 = hc.createDataFrame(df2_pd)
#df1.show()

In [14]:
df1.printSchema()

root
 |-- Gender: string (nullable = true)
 |-- PassengerId: long (nullable = true)
 |-- Name: string (nullable = true)
 |-- Survived: long (nullable = true)



#### With Defined Data Schema 

In [17]:
df_schema_1 = StructType([StructField('Gender',StringType(),True),
                          StructField('PassengerId',IntegerType(),True),
                          StructField('Name',StringType(),True),
                          StructField('Survived',IntegerType(),True) ])
# for date type: StructField('MeetingDate',DateType(),True) ])
#True denotes that data field is nullable
df_spark_1 = hc.createDataFrame(df1_pd,df_schema_1)

#### Repeat the same for df2_pd

#### 2.Using Hadoop Tables:


In [ ]:
# By importing the whole table: hc.table('table name')
#cust_1: database name in hadoop; #abc_xxx : table name
data = hc.table('cust_1.abc_xxx') 
# data is now a pyspark dataframe

# By running a sql query:

data = hc.sql(''' select name, r_no, age from cust_1.abc_xxx where trim(age) = '30' ''')

#data is now a pyspark dataframe

# To Note: trim is used in string types to ensure removal of white spaces, since data in hive can be unstructured

#### 3.Loading From CSV

We can load our data from a CSV file in an S3 bucket. There are three ways to handle data types (dtypes) for each column: The easiest, but the most computationally-expensive, is to pass inferSchema=True to the load method. The second way entails specifiying the dtypes manually for every column by passing schema=StructType(...), which is computationally-efficient but may be difficult and prone to coder error for especially wide datasets. The final option is to not specify a schema option at all, in which case Spark will assign all the columns string dtypes. Note that dtypes can be changed later, as we will demonstrate, though it is more costly than doing it correctly in the loading process.

Loading the data with the schema inferred:

Note: Check this link for the file: https://github.com/UrbanInstitute/pyspark-tutorials/blob/master/01_pyspark-basics-1.ipynb

In [ ]:
df = spark.read.format('com.databricks.spark.csv').options(header='False', inferschema='true', sep='|').\
load('s3://ui-spark-social-science-public/data/Performance_2015Q1.txt')


Example loading of the same data by passing a custom schema:

In [ ]:

"""
 from pyspark.sql.types import DateType, TimestampType, IntegerType, FloatType, LongType, DoubleTypefrom py 
from pyspark.sql.types import StructType, StructField

custom_schema = StructType([StructField('_c0', DateType(), True),
                           StructField('_c1', StringType(), True),
                           StructField('_c2', DoubleType(), True),
                           StructField('_c3', DoubleType(), True),
                           StructField('_c4', DoubleType(), True),
                           StructField('_c5', IntegerType(), True),
                           ...
                           StructField('_c27', StringType(), True)])
                           
df = spark.read.csv('s3://ui-spark-social-science-public/data/Performance_2015Q1.txt', header=False, schema=custom_schema, sep='|')
""";

One example of using infering and specifying a schema together might be with a large, unfamiliar dataset that you know you will need to load up and work with repeatedly. The first time you load it use inferSchema, then make note of the dtypes it assigns. Use that information to build the custom schema, so that when you load the data in the future you avoid the extra processing time necessary for infering.

#### Interesting read to use persist() to improve computation speed by branching : https://github.com/UrbanInstitute/pyspark-tutorials/blob/master/02_pyspark-basics-2.ipynb

https://github.com/UrbanInstitute/pyspark-tutorials

#### Use the titanic data going further

### 2. Change DataFrames Properties
#### 2.1 Change Column Type
 Available types include <br>

BinaryType <br>
BooleanType <br>
ByteType <br>
DoubleType <br>
DateType <br>
FloatType <br>
IntegerType <br>
etc.

#### Q. Change data type of PassengerID to String

In [19]:
from pyspark.sql.types import IntegerType, DateType

In [21]:
# .withColumn return a DataFrame by adding a column or replacing the existing column that has the same name.
df_spark_2 = df_spark_1.withColumn("PassengerId", df_spark_1["PassengerId"].cast(StringType()))


In [22]:
df_spark_2.printSchema()

root
 |-- Gender: string (nullable = true)
 |-- PassengerId: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Survived: integer (nullable = true)



In [23]:
df_spark_2.dtypes

[('Gender', 'string'),
 ('PassengerId', 'string'),
 ('Name', 'string'),
 ('Survived', 'int')]

#### 2.2 Change Column Name

#### Q. Rename Gender Column to Sex and Name to Passenger_Name

In [52]:
df_spark_3 = df_spark_2.withColumnRenamed("Gender", "Sex").withColumnRenamed("Name", "Passenger_Name")

In [53]:
df_spark_3.printSchema()

root
 |-- Sex: string (nullable = true)
 |-- PassengerId: string (nullable = true)
 |-- Passenger_Name: string (nullable = true)
 |-- Survived: integer (nullable = true)



In [ ]:
df_spark_3 = df_spark_3

#### 2.3 Modify Column Values and create new columns

#### Q1. Modify values in Sex Column: M to Male and F to female

In [29]:
df_spark_4 = df_spark_3.withColumn("Sex", F.when(trim(df_spark_3['Sex']) == 'M', 'Male').\
                                   otherwise(F.when(trim(df_spark_3['Sex']) == 'F', 'Female').otherwise('NA')))

In [39]:
df_spark_4

DataFrame[Sex: string, PassengerId: string, Name: string, Survived: int]



#### Q2. Create New column 'type_of_data' with constant value: 'train'

In [37]:
df_spark_5 = df_spark_4.withColumn('type_of_data', lit('train'))

#### 2.4 Order DataFrame by Specified Column(s)

#### Q. Order by  descending order: PassengerId

In [31]:
df_spark_4.sort(df_spark_4.PassengerId.desc()).show()

+------+-----------+--------+--------+
|   Sex|PassengerId|    Name|Survived|
+------+-----------+--------+--------+
|  Male|          5| William|       0|
|Female|          4|    Lily|       1|
|Female|          3|   Laina|       1|
|Female|          2|Florence|       1|
|  Male|          1|    Owen|       0|
+------+-----------+--------+--------+



In [40]:
df_spark_4.sort('PassengerId', ascending=False).show()

+------+-----------+--------+--------+
|   Sex|PassengerId|    Name|Survived|
+------+-----------+--------+--------+
|  Male|          5| William|       0|
|Female|          4|    Lily|       1|
|Female|          3|   Laina|       1|
|Female|          2|Florence|       1|
|  Male|          1|    Owen|       0|
+------+-----------+--------+--------+



#### 2.4 Filter by Specified Column(s)

#### Q. Filter out the female population from the data

Filter takes column expression or SQL expression ---


1.Using Column expression

In [36]:
female_data = df_spark_4.filter(trim(col('Sex')) == 'Female')

####  col('columnname') is eqivalent to dataframe['columnname'] equivalent to dataframe.columnname

2.Using SQL expression.Note the double and single quotes

In [34]:
female_data_2 = df_spark_4.filter("Sex='Female'")

#### 2.5 Summarize/Aggregate and group by

#### Q. Group By Passenger Class and find avergae Fare and Age

In [42]:
gdf2 = df2.groupby('Pclass')

avg_cols = ['Age', 'Fare']
gdf2.avg(*avg_cols).show()

+------+------------------+-----------------+
|Pclass|          avg(Age)|        avg(Fare)|
+------+------------------+-----------------+
|     1|              36.5|             62.2|
|     3|27.666666666666668|7.733333333333333|
+------+------------------+-----------------+



#### Q. Group By Passenger Class and find Total Fare and Average Age and count of passengers

In [43]:
gdf2.agg({'*': 'count', 'Age': 'avg', 'Fare':'sum'}).show()

+------+--------+------------------+---------+
|Pclass|count(1)|          avg(Age)|sum(Fare)|
+------+--------+------------------+---------+
|     1|       2|              36.5|    124.4|
|     3|       3|27.666666666666668|     23.2|
+------+--------+------------------+---------+



#### Q. rename the columns count(1), avg(Age) etc, 
using toDF().

In [44]:
gdf2.agg({'*': 'count', 'Age': 'avg', 'Fare':'sum'})\
    .toDF('Pclass', 'counts', 'average_age', 'total_fare')\
    .show()

+------+------+------------------+----------+
|Pclass|counts|       average_age|total_fare|
+------+------+------------------+----------+
|     1|     2|              36.5|     124.4|
|     3|     3|27.666666666666668|      23.2|
+------+------+------------------+----------+



#### Q. Arrange passenger class by total fare in ascending

In [46]:
df2.groupby('Pclass').agg({'Fare':'sum'}).sort('sum(Fare)', ascending = True).show()

+------+---------+
|Pclass|sum(Fare)|
+------+---------+
|     3|     23.2|
|     1|    124.4|
+------+---------+



#### Q. Get Count of distinct Passenger classes

In [47]:
df2.select('Pclass').distinct().count()

2

In [48]:
df2.select('Pclass').distinct().show()

+------+
|Pclass|
+------+
|     1|
|     3|
+------+



#### Q. Get Count of 'Distinct' Passenger Ids by Sex

#####Note 1: countDistinct here is a SQL Function hence 'F.countDistinct'; <br>
#####Note 2: Like F.countDistinct, F.sum, F.avg, F.max, etc sql functions can be used. <br>
#####Note 3: here there is no repitition of passenger ids hence count gives same result as count distinct function

In [49]:
df_spark_4.groupby('Sex').agg(F.countDistinct('PassengerId')).show()

+------+------------------+
|   Sex|count(PassengerId)|
+------+------------------+
|Female|                 3|
|  Male|                 2|
+------+------------------+



In [50]:
df2.groupby('Pclass').agg(F.sum('Fare')).show()

+------+---------+
|Pclass|sum(Fare)|
+------+---------+
|     1|    124.4|
|     3|     23.2|
+------+---------+



###Joins and unions¶
####There are two ways to combine dataframes --- joins and unions. The idea here is the same as joining and unioning tables in SQ

####Joins¶ Q.Join the two titanic dataframes by the column PassengerId

In [51]:
df1.join(df2, ['PassengerId']).show()

+-----------+------+--------+--------+----+---+------+
|PassengerId|Gender|    Name|Survived|Fare|Age|Pclass|
+-----------+------+--------+--------+----+---+------+
|          1|     M|    Owen|       0| 7.3| 22|     3|
|          2|     F|Florence|       1|71.3| 38|     1|
|          3|     F|   Laina|       1| 7.9| 26|     3|
|          4|     F|    Lily|       1|53.1| 35|     1|
|          5|     M| William|       0| 8.0| 35|     3|
+-----------+------+--------+--------+----+---+------+



In [54]:
df1.join(df2, trim(df1['PassengerId']) == trim(df2['PassengerId']), 'left_outer').show()

+------+-----------+--------+--------+----+---+-----------+------+
|Gender|PassengerId|    Name|Survived|Fare|Age|PassengerId|Pclass|
+------+-----------+--------+--------+----+---+-----------+------+
|     M|          1|    Owen|       0| 7.3| 22|          1|     3|
|     F|          2|Florence|       1|71.3| 38|          2|     1|
|     F|          3|   Laina|       1| 7.9| 26|          3|     3|
|     F|          4|    Lily|       1|53.1| 35|          4|     1|
|     M|          5| William|       0| 8.0| 35|          5|     3|
+------+-----------+--------+--------+----+---+-----------+------+



#### Note : Joins using condition create duplicate columns with same name; it creates problem later while storing back to hadoop database , etc hence better to drop one of it

In [55]:
df1.join(df2, trim(df1['PassengerId']) == trim(df2['PassengerId']), 'left_outer').drop(df2['PassengerId']).show()

+------+-----------+--------+--------+----+---+------+
|Gender|PassengerId|    Name|Survived|Fare|Age|Pclass|
+------+-----------+--------+--------+----+---+------+
|     M|          1|    Owen|       0| 7.3| 22|     3|
|     F|          2|Florence|       1|71.3| 38|     1|
|     F|          3|   Laina|       1| 7.9| 26|     3|
|     F|          4|    Lily|       1|53.1| 35|     1|
|     M|          5| William|       0| 8.0| 35|     3|
+------+-----------+--------+--------+----+---+------+



### Nonequi joins

Here is an example of nonequi join. They can be very slow due to skewed data, but this is one operation that Spark can do while Hive can not.

In [56]:
df1.join(df2, df1.PassengerId <= df2.PassengerId).show() # Note the duplicate col names

+------+-----------+--------+--------+----+---+-----------+------+
|Gender|PassengerId|    Name|Survived|Fare|Age|PassengerId|Pclass|
+------+-----------+--------+--------+----+---+-----------+------+
|     M|          1|    Owen|       0| 7.3| 22|          1|     3|
|     M|          1|    Owen|       0|71.3| 38|          2|     1|
|     F|          2|Florence|       1|71.3| 38|          2|     1|
|     M|          1|    Owen|       0| 7.9| 26|          3|     3|
|     M|          1|    Owen|       0|53.1| 35|          4|     1|
|     M|          1|    Owen|       0| 8.0| 35|          5|     3|
|     F|          2|Florence|       1| 7.9| 26|          3|     3|
|     F|          2|Florence|       1|53.1| 35|          4|     1|
|     F|          2|Florence|       1| 8.0| 35|          5|     3|
|     F|          3|   Laina|       1| 7.9| 26|          3|     3|
|     F|          3|   Laina|       1|53.1| 35|          4|     1|
|     F|          3|   Laina|       1| 8.0| 35|          5|   

###Unions
Union() returns a dataframe from the union of two dataframes

In [58]:
df1.unionAll(df1).show()

+------+-----------+--------+--------+
|Gender|PassengerId|    Name|Survived|
+------+-----------+--------+--------+
|     M|          1|    Owen|       0|
|     F|          2|Florence|       1|
|     F|          3|   Laina|       1|
|     F|          4|    Lily|       1|
|     M|          5| William|       0|
|     M|          1|    Owen|       0|
|     F|          2|Florence|       1|
|     F|          3|   Laina|       1|
|     F|          4|    Lily|       1|
|     M|          5| William|       0|
+------+-----------+--------+--------+



 Similarly, df1.intersect(df2) can be used to get intersection

 ###  4. Transform A DataFrame Column (using UDF)

In [63]:
from pyspark.sql.functions import udf

derive_perc = udf(lambda x: "$" + str(x * 100) )
# or 
# @udf
# def derive_perc(x):
#     return(str(x * 100, 3) + "%")

thrice_fare = df2.withColumn("fare_3times", derive_perc(df2['Fare']))

thrice_fare.show()

+----+---+-----------+------+-----------+
|Fare|Age|PassengerId|Pclass|fare_3times|
+----+---+-----------+------+-----------+
| 7.3| 22|          1|     3|     $730.0|
|71.3| 38|          2|     1|    $7130.0|
| 7.9| 26|          3|     3|     $790.0|
|53.1| 35|          4|     1|    $5310.0|
| 8.0| 35|          5|     3|     $800.0|
+----+---+-----------+------+-----------+



### 5. Using Spark SQL
Reference: https://github.com/XD-DENG/Spark-practice/blob/master/Spark%20DataFrames%20%26%20SQL%20-%20Basics.ipynb

We can build a view with the Spark DataFrame, then we can SQL syntax to further process our data.

You may notice there are two ways to build a view,

DF.createGlobalTempView (or DF.createOrReplaceGlobalTempView): create a global temporary view
DF.createTempView (or DF.createOrReplaceTempView): create a local temporary view
The main difference between them is the lifetime. The lifetime of a global temporary view is tied to the Spark application, while lifetime of a local temporary view is tied to the SparkSession.

In [64]:

# Creates or replaces a local temporary view with a DataFrame.# Create 
# The lifetime of this temporary table is tied to the SparkSession that was used to create this DataFrame.
package_count.createOrReplaceTempView("package_count_sql_table")



NameError: name 'package_count' is not defined

In [ ]:
# Basic Spark SQL Query - 1
query_result = sqlContext.sql("select perc \
                               from package_count_sql_table \
                               where package = 'DT'")

print(query_result.collect())

In [ ]:
# Basic Spark SQL Query - 2
query_result = sqlContext.sql("select * \
                                from package_count_sql_table \
                                where count > 1000 \
                                order by count asc")
print(query_result.show(5))

In [ ]:

# Use the Spark RDD way to process the results from Spark SQL query result# Use th 
query_result.rdd.map(lambda x:x['package'] + ":" + x['perc']).take(10)

#### registering the dataframe as table in hadoop to use in select queries; act as temp tables

In [ ]:
 hc.registerDataFrameAsTable(query_result, 'query_result_tbl')

In [ ]:
test = hc.sql('query_result_tbl')

#### to register /store as permamnent table

In [ ]:
query_result.write.option("sep","|").format('orc').mode("overwrite").saveAsTable("Cust_xxx.nxxx")